<a href="https://colab.research.google.com/github/charan06k/verriassist/blob/main/drug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch as tr
import pandas as pd
import numpy as np
import sys
file_path='/content/drug_prescription.xls'
full_data=pd.read_excel(file_path)

In [2]:
# Converting categorical data to numerical values
for index, row in full_data.iterrows():
    if row['Sex'] == 'M':
        full_data.at[index, 'Sex'] = 1
    elif row['Sex'] == 'F':
        full_data.at[index, 'Sex'] = -1

    if row['BP'] == 'LOW':
        full_data.at[index, 'BP'] = -1
    elif row['BP'] == 'NORMAL':
        full_data.at[index, 'BP'] = 0
    elif row['BP'] == 'HIGH':
        full_data.at[index, 'BP'] = 1

    if row['Cholesterol'] == 'LOW':
        full_data.at[index, 'Cholesterol'] = -1
    elif row['Cholesterol'] == 'NORMAL':
        full_data.at[index, 'Cholesterol'] = 0
    elif row['Cholesterol'] == 'HIGH':
        full_data.at[index, 'Cholesterol'] = 1

    if row['Drug'] == 'drugA':
        full_data.at[index, 'Drug'] = 0
    elif row['Drug'] == 'drugB':
        full_data.at[index, 'Drug'] = 1
    elif row['Drug'] == 'drugC':
        full_data.at[index, 'Drug'] = 2
    elif row['Drug'] == 'drugX':
        full_data.at[index, 'Drug'] = 3
    elif row['Drug'] == 'DrugY':
        full_data.at[index, 'Drug'] = 4

    full_data.at[index, 'Age'] = int(row['Age'])  # explicitly convert age column to int
    full_data.at[index, 'Na_to_K'] = float(row['Na_to_K'])  # explicitly convert Na_to_K column to float


# Convert specific columns to desired data types
full_data['Sex'] = full_data['Sex'].astype(int)
full_data['BP'] = full_data['BP'].astype(int)
full_data['Cholesterol'] = full_data['Cholesterol'].astype(int)
full_data['Drug'] = full_data['Drug'].astype(int)

age = full_data[["Age"]]
sex = full_data[["Sex"]]
bp = full_data[["BP"]]
ch = full_data[["Cholesterol"]]
n_k = full_data[["Na_to_K"]]
drug = full_data[["Drug"]]
drug_pred = None
error = None
print(age)
print(sex)
print(bp)
print(ch)
print(n_k)
print(drug)

     Age
0     23
1     47
2     47
3     28
4     61
..   ...
195   56
196   16
197   52
198   23
199   40

[200 rows x 1 columns]
     Sex
0     -1
1      1
2      1
3     -1
4     -1
..   ...
195   -1
196    1
197    1
198    1
199   -1

[200 rows x 1 columns]
     BP
0     1
1    -1
2    -1
3     0
4    -1
..   ..
195  -1
196  -1
197   0
198   0
199  -1

[200 rows x 1 columns]
     Cholesterol
0              1
1              1
2              1
3              1
4              1
..           ...
195            1
196            1
197            1
198            0
199            0

[200 rows x 1 columns]
     Na_to_K
0     25.355
1     13.093
2     10.114
3      7.798
4     18.043
..       ...
195   11.567
196   12.006
197    9.894
198   14.020
199   11.349

[200 rows x 1 columns]
     Drug
0       4
1       2
2       2
3       3
4       4
..    ...
195     2
196     2
197     3
198     3
199     3

[200 rows x 1 columns]


In [3]:

print(age.shape)
print(sex.shape)
print(bp.shape)
print(ch.shape)
print(n_k.shape)

(200, 1)
(200, 1)
(200, 1)
(200, 1)
(200, 1)


In [4]:
#assigning weights
import random
w1=random.random()
w2=random.random()
w3=random.random()
w4=random.random()
w5=random.random()
print(w1)
print(w2)
print(w3)
print(w4)
print(w5)

0.7311831366366633
0.6758942825377288
0.8631473579233127
0.0873748207561793
0.43897525031777795


In [5]:
def calculate_error(drug_pred, drug):
       error = drug - drug_pred
       mse = np.mean(error**2)  # Mean Squared Error
       return mse

In [6]:
def calculate_gradient(w1, w2, w3, w4, w5, age, sex, bp, ch, n_k, drug):
    # Get the underlying NumPy arrays from the DataFrames
    age_values = age.values
    sex_values = sex.values
    bp_values = bp.values
    ch_values = ch.values
    n_k_values = n_k.values
    drug_values = drug.values


    # Calculate drug_pred using NumPy array operations
    drug_pred = (w1 * age_values) + (w2 * sex_values) + (w3 * bp_values) + (w4 * ch_values) + (w5 * n_k_values)
    error = drug_values - drug_pred

    # Partial derivatives (gradients) - use NumPy's dot product for efficiency
    grad_w1 = -2 * np.dot(error.T, age_values) / len(drug)  # Assuming MSE loss
    grad_w2 = -2 * np.dot(error.T, sex_values) / len(drug)
    grad_w3 = -2 * np.dot(error.T, bp_values) / len(drug)
    grad_w4 = -2 * np.dot(error.T, ch_values) / len(drug)
    grad_w5 = -2 * np.dot(error.T, n_k_values) / len(drug)

    return grad_w1, grad_w2, grad_w3, grad_w4, grad_w5,drug_pred,error

In [14]:
learning_rate = 0.01
count = 1000
i = 0
for i in range(count):
       grad_w1, grad_w2, grad_w3, grad_w4, grad_w5, drug_pred, error = calculate_gradient(w1, w2, w3, w4, w5, age, sex, bp, ch, n_k, drug)

       # Update weights
       w1 = w1 - float(learning_rate) * grad_w1
       w2 = w2 - float(learning_rate) * grad_w2
       w3 = w3 - float(learning_rate) * grad_w3
       w4 = w4 - float(learning_rate) * grad_w4
       w5 = w5 - float(learning_rate) * grad_w5

print(w1,w2,w3,w4,w5)
print(grad_w1.dtype)


[[nan]] [[nan]] [[nan]] [[nan]] [[nan]]
float64


In [ ]:
age_test=int(input("enter the age: "))
sex_test=input("enter sex as \"M\" or \"F\": ")
bp_test=input("enter BP as \"HIGH\" or \"LOW\" or \"NORMAL\": ")
ch_test=input("enter cholestrol as \"HIGH\" or \"LOW\" or \"NORMAL\": ")
n_k_test=float(input("enter the Na to k ratio : "))

#converting data to required form
if sex_test=='M':
  sex_test=1
elif sex_test=='F':
  sex_test=-1

if bp_test=='HIGH':
  bp_test=1
elif bp_test=='LOW':
  bp_test=-1
elif bp_test=='NORMAL':
  bp_test=0

if ch_test=='HIGH':
  ch_test=1
elif ch_test=='LOW':
  ch_test=-1
elif ch_test=='NORMAL':
  ch_test=0

drug_prec=(w1*age_test)+(w2*sex_test)+(w3*bp_test)+(w4*ch_test)+(w5*n_k_test)

if -0.5<=drug_prec<=0.5:
  drug_prec='DrugA'
elif 0.5<=drug_prec<=1.5:
  drug_prec='DrugB'
elif 1.5<=drug_prec<=2.5:
  drug_prec='DrugC'
elif 2.5<=drug_prec<=3.5:
  drug_prec='DrugX'
elif 3.5<=drug_prec<=4.5:
  drug_prec='DrugY'
else:
  print(f"wrond data entered")
print(f"precribed drug :", drug_prec)

enter the age: 23
enter sex as "M" or "F": M
enter BP as "HIGH" or "LOW" or "NORMAL": HIGH
enter cholestrol as "HIGH" or "LOW" or "NORMAL": NORMAL
enter the Na to k ratio : 10.234
wrond data entered
precribed drug : [[nan]]
